In [ ]:
import pandas as pd
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
import sys
sys.path.append("../")

from utils.io import read_parameters
from statannot import add_stat_annotation

In [ ]:
# read parameters and key file

parameter_file  = "../local/parameters.yml"
parameters = read_parameters(parameter_file)       

key_file_path = parameters["key_file"]
key_file = pd.read_csv(key_file_path)

print("key file columns:", key_file.columns)
print("key file conditions:", key_file["condition"].unique())

In [ ]:
# parameter_file  = "../local/parameters.yml"
# parameters = read_parameters(parameter_file)   

# base_folder = parameters["base_folder"]
# tracking_data_files = parameters["tracking_data"]
# data_exclude = parameters["data_exclude"]
output_folder = parameters["output_folder"]
tracking_data_path = parameters["output_folder"] + "/speed_data/"

# single_time_point_evaluation = parameters["single_time_point_evaluation"]

interval = parameters["time_lag"]

errorbar = ("ci",95)

plt.rcParams.update({'font.size': 14})
interval = parameters["time_lag"]

subsample_n = 10 # for speed up of draft plotting
subsample_frac = 0.1 # for speed up of draft plotting

In [ ]:
## Cdc42 manuscript

#hue_order = ["siScr", "siCdc42", "siRac1"]
# condition = "mosaic_siScr_siCdc42"
# treatment_order = ["siScr", "siCdc42"]
# color_palette = {"siScr": "limegreen", "siCdc42": "violet"}

condition = "mosaic_siScr_siRac1"
treatment_order = ["siScr", "siRac1"]
color_palette = {"siScr": "limegreen", "siRac1": "violet"}

# condition = "mosaic_siScr_siScr"
# treatment_order = ["siScr"]
# color_palette = {"green": "limegreen", "red": "violet"}

## SMAD4 manuscript

#condition = "mosaic_siCTRL_siCTRL"
#treatment_order = ["siCTRL"]

#condition = "mosaic_siCTRL_siSMAD4"
#treatment_order = ["siCTRL", "siSMAD4"]

#condition = "mosaic_siCTRL_siAlk1"
#treatment_order = ["siCTRL", "siAlk1"]

key_selection = key_file[key_file["condition"] == condition]
display(key_selection) 

key_selection = key_selection[key_selection["experimentID"] != "240301_siScr_1_mosaic"]
display(key_selection) 

In [ ]:
parameters

# load data

In [ ]:
migration_speed_df = pd.DataFrame()

for index, row in key_selection.iterrows():
    print("processing", row["condition"])
    #tracking_data_file = tracking_data_path + row["filename"]
    tracking_data_file = output_folder + "speed_data/migration_speed_df_%s_%s_%s.csv" % (row["treatment"],
                                                                                         row["color"], 
                                                                                         row["experimentID"])
    tracking_data = pd.read_csv(tracking_data_file)

    tracking_data['color'] = row['color']
    tracking_data['treatment'] = row['treatment']
    tracking_data['experimentID'] = row['experimentID']    

    if migration_speed_df.empty:
        migration_speed_df = tracking_data
    else:    
        migration_speed_df = pd.concat([migration_speed_df, tracking_data], ignore_index=True)


#migration_speed_df = migration_speed_df[migration_speed_df["time_lag"] == interval]
migration_speed_df

In [ ]:
full_hours = np.arange(0, migration_speed_df["FRAME"].max(), interval)
print(full_hours)
migration_speed_df = migration_speed_df[migration_speed_df["FRAME"].isin(full_hours)]
migration_speed_df

# velocity against flow grouped by condition and filename

In [ ]:
plt.rcParams.update({'font.size': 14})

plot_migration_speeds = migration_speed_df.dropna()
plot_migration_speeds = plot_migration_speeds.iloc[::subsample_n, :]
#plot_migration_speeds = plot_migration_speeds.sample(frac=subsample_frac, replace=True, random_state=1)
plot_migration_speeds = plot_migration_speeds[plot_migration_speeds["FRAME"] >= interval]
print("plot_migration_speeds", plot_migration_speeds.shape)

#fig, ax = plt.subplots(len(intervals),figsize=(15,30))
fig, ax = plt.subplots(1, figsize=(16,8))

sns.lineplot(x = "time_in_h", y = "vel_x_mu_per_h", hue = "treatment", hue_order = treatment_order, 
             data = plot_migration_speeds, style="filename", ax=ax, errorbar = errorbar) 

#sns.lineplot(x = "FRAME", y = "vel_x", hue = "condition", data = plot_migration_speeds, ax=ax, ci = 90) 
#interval_length_min = 5*intervals[i]
#ax.set_title("time interval %s min" % interval_length_min)
ax.set_ylabel("velocity parallel to flow in microns/h")
#ax.set_ylim(-7.0,7.0)
ax.axhline(y = 0.0, color = 'r', linestyle = 'dashed')
plt.legend(bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0.)
plt.tight_layout()
plt.savefig("velocity_parallel_to_flow_filename_all.pdf")
plt.savefig("velocity_parallel_to_flow_filename_all.png")

# velocity against flow grouped by condition

In [ ]:
plt.rcParams.update({'font.size': 14})

plot_migration_speeds = migration_speed_df.dropna()
#plot_migration_speeds = plot_migration_speeds.iloc[::subsampling_n, :]
#plot_migration_speeds = plot_migration_speeds.sample(frac=subsample_frac, replace=True, random_state=1)
plot_migration_speeds = plot_migration_speeds[plot_migration_speeds["FRAME"] >= 2*interval]


#fig, ax = plt.subplots(len(intervals),figsize=(15,30))
fig, ax = plt.subplots(1, figsize=(8,8))

#sns.lineplot(x = "time_in_h", y = "vel_x_mu_per_h", hue = "condition", data = plocolor_palette = {"siScr": "limegreen", "siCdc42": "violet"}t_migration_speeds, ax=ax, ci = 90) 

if len(treatment_order) > 1: 
    sns.lineplot(x = "time_in_h", y = "vel_x_mu_per_h", hue = "treatment", hue_order = treatment_order, 
             data = plot_migration_speeds, ax=ax, errorbar = errorbar, palette = color_palette) 
else:
    sns.lineplot(x = "time_in_h", y = "vel_x_mu_per_h", hue = "color", hue_order = ["red", "green"],
             data = plot_migration_speeds, ax=ax, errorbar = errorbar, palette=color_palette) 


#sns.lineplot(x = "FRAME", y = "vel_x", hue = "condition", data = plot_migration_speeds, ax=ax, ci = 90) 
#interval_length_min = 5*intervals[i]
#ax.set_title("time interval %s min" % interval_length_min)
ax.set_ylabel("velocity parallel to flow in $\mu$m/h")
ax.set_ylim(-5.0,5.0)
#plt.legend(bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0.)
ax.set_xlabel("time in h")
#ax.set_xlim(0,65)
#plt.tight_layout()
ax.axhline(y = 0.0, color = 'r', linestyle = 'dashed')
plt.savefig("velocity_parallel_to_flow_%s.pdf" % condition)
plt.savefig("velocity_parallel_to_flow_%s.png" % condition)

In [ ]:
plot_migration_speeds = migration_speed_df.dropna()
#plot_migration_speeds = plot_migration_speeds.iloc[::subsampling_n, :]
#plot_migration_speeds = plot_migration_speeds.sample(frac=subsample_frac, replace=True, random_state=1)
plot_migration_speeds = plot_migration_speeds[plot_migration_speeds["FRAME"] >= interval]


plt.rcParams.update({'font.size': 14})

#fig, ax = plt.subplots(len(intervals),figsize=(15,30))
fig, ax = plt.subplots(1, figsize=(8,8))

#sns.lineplot(x = "time_in_h", y = "vel_x_mu_per_h", hue = "condition", data = plot_migration_speeds, ax=ax, ci = 90) 

# sns.lineplot(x = "time_in_h", y = "vel_mu_per_h", hue = "condition", 
#              hue_order = hue_order, data = plot_migration_speeds, ax=ax, errorbar = errorbar) 
if len(treatment_order) > 1: 
    sns.lineplot(x = "time_in_h", y = "vel_mu_per_h", hue = "treatment", hue_order = treatment_order, 
            data = plot_migration_speeds, ax=ax, errorbar = errorbar, palette = color_palette)
else:            
    sns.lineplot(x = "time_in_h", y = "vel_mu_per_h",  hue = "color", hue_order = ["green", "red"],
            data = plot_migration_speeds, ax=ax, errorbar = errorbar, palette = color_palette) 

#sns.lineplot(x = "FRAME", y = "vel_x", hue = "condition", data = plot_migration_speeds, ax=ax, ci = 90) 
#interval_length_min = 5*intervals[i]
#ax.set_title("time interval %s min" % interval_length_min)
ax.set_ylabel("velocity in microns/h")
ax.set_xlabel("time in h")
ax.set_ylim(0.0,17.0)
#plt.legend(bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0.)
#plt.tight_layout()
ax.axhline(y = 0.0, color = 'r', linestyle = 'dashed')
plt.savefig("velocity_%s.pdf" % condition)
plt.savefig("velocity_%s.png" % condition)

In [ ]:
plt.rcParams.update({'font.size': 14})

plot_migration_speeds = migration_speed_df.dropna()
plot_migration_speeds = plot_migration_speeds[plot_migration_speeds["FRAME"] >= interval]



#fig, ax = plt.subplots(len(intervals),figsize=(15,30))
fig, ax = plt.subplots(1, figsize=(16,8))

sns.lineplot(x = "time_in_h", y = "vel_mu_per_h", hue = "condition", hue_order = hue_order, 
             data = plot_migration_speeds, style="filename", ax=ax, errorbar = errorbar) 

#sns.lineplot(x = "FRAME", y = "vel_x", hue = "condition", data = plot_migration_speeds, ax=ax, ci = 90) 
#interval_length_min = 5*intervals[i]
#ax.set_title("time interval %s min" % interval_length_min)
ax.set_ylabel("velocity against flow in microns/h")
ax.set_ylim(0.0,12.0)
ax.axhline(y = 0.0, color = 'r', linestyle = 'dashed')
plt.legend(bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0.)
plt.tight_layout()
plt.savefig("velocity_filename_filtered.pdf")
plt.savefig("velocity_filename_filtered.png")

In [ ]:
plot_migration_speeds = migration_speed_df.dropna()
plot_migration_speeds = plot_migration_speeds[plot_migration_speeds["FRAME"] >= interval]



#fig, ax = plt.subplots(len(intervals),figsize=(15,30))
fig, ax = plt.subplots(1, figsize=(16,8))

sns.lineplot(x = "time_in_h", y = "vel_y_mu_per_h", hue = "condition", hue_order = hue_order, 
             data = plot_migration_speeds, style="filename", ax=ax, errorbar = errorbar) 

#sns.lineplot(x = "FRAME", y = "vel_x", hue = "condition", data = plot_migration_speeds, ax=ax, ci = 90) 
#interval_length_min = 5*intervals[i]
#ax.set_title("time interval %s min" % interval_length_min)
ax.set_ylabel("velocity perpendicular to flow in microns/h")
ax.set_ylim(-7.0,7.0)
ax.axhline(y = 0.0, color = 'r', linestyle = 'dashed')
plt.legend(bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0.)
plt.tight_layout()
plt.savefig("velocity_perpendicular_to_flow_filename_filtered.pdf")
plt.savefig("velocity_perpendicular_to_flow_filename_filtered.png")

In [ ]:
plot_migration_speeds = migration_speed_df.dropna()
plot_migration_speeds = plot_migration_speeds[plot_migration_speeds["FRAME"] >= interval]
plot_migration_speeds = plot_migration_speeds[plot_migration_speeds["filename"] != "/siScr/SUM_230420_siScr_20dyn_TrackMate.csv"]
plot_migration_speeds = plot_migration_speeds[plot_migration_speeds["filename"] != "/siRac1/SUM_230420_siRac1_TrackMate.csv"]

#fig, ax = plt.subplots(len(intervals),figsize=(15,30))
fig, ax = plt.subplots(1, figsize=(15,10))

sns.lineplot(x = "time_in_h", y = "vel_y_mu_per_h", hue = "condition", 
             data = plot_migration_speeds, ax=ax, errorbar = errorbar) 

#sns.lineplot(x = "FRAME", y = "vel_y", hue = "condition", data = plot_migration_speeds, ax=ax, ci = 90) 
#interval_length_min = 5*intervals[i]
#ax.set_title("time interval %s min" % interval_length_min)
ax.set_ylabel("velocity in microns/h")
ax.set_ylim(-7.0,7.0)
ax.axhline(y = 0.0, color = 'r', linestyle = 'dashed')

plt.savefig("velocity_perpendicular_to_flow.pdf")
plt.savefig("velocity_perpendicular_to_flow.png")

In [ ]:
single_time_point = plot_migration_speeds[plot_migration_speeds["FRAME"]==24]
#single_time_point.groupby(["filename","condition"])["vel_x"].mean()

display(single_time_point)
plot_data = single_time_point.groupby(["condition", "filename"])["vel_x_mu_per_h"].mean().reset_index()
display(plot_data)

In [ ]:
fig, ax = plt.subplots(1, figsize=(10,10))

plt.rcParams.update({'font.size': 14})

plot_migration_speeds = migration_speed_df.dropna()
single_time_point = plot_migration_speeds[plot_migration_speeds["FRAME"]==22]
plot_data = single_time_point.groupby(["condition", "filename"])["vel_x_mu_per_h"].mean().reset_index()
#plot_data = single_time_point.copy()

sns.boxplot(data = plot_data, x = "condition", order = hue_order, y = "vel_x_mu_per_h" )
sns.swarmplot(data = plot_data, x = "condition", order = hue_order, y = "vel_x_mu_per_h" , color="black", size =10)
test_results = add_stat_annotation(ax, data=plot_data, y = "vel_x_mu_per_h", x = "condition", order = hue_order, 
                                   #order=order,
                                   box_pairs=[("siScr", "siCdc42"), ("siScr", "siRac1"), ("siCdc42", "siRac1")],
                                   test='t-test_welch', 
                                   #text_format='full',
                                   #text_format='star',
                                   text_format='simple',
                                   loc='inside', verbose=2)

ax.set_ylabel("velocity against flow in microns/h")
ax.set_title("velocity against flow after 2h")
ax.set_ylim(-7.5,0.5)

plt.savefig("velocity_against_flow_box_plot.png")
plt.savefig("velocity_against_flow_box_plot.pdf")
#plt.legend(bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0.)
#plt.legend(loc=2, borderaxespad=0.)

In [ ]:
fig, ax = plt.subplots(1, figsize=(10,10))

plt.rcParams.update({'font.size': 14})

plot_migration_speeds = migration_speed_df.dropna()

plot_migration_speeds = plot_migration_speeds.iloc[::10, :]

single_time_point = plot_migration_speeds[plot_migration_speeds["FRAME"]==24]
#plot_data = single_time_point.groupby(["condition", "filename"])["vel_x_mu_per_h"].mean().reset_index()
plot_data = single_time_point.copy()

#sns.boxplot(data = plot_data, x = "condition", order = hue_order, y = "vel_x_mu_per_h" )
sns.swarmplot(data = plot_data, x = "condition", order = hue_order, y = "vel_x_mu_per_h" , color="black")#, size =10)
sns.violinplot(data = plot_data, x = "condition", order = hue_order, y = "vel_x_mu_per_h")#, size =10)


test_results = add_stat_annotation(ax, data=plot_data, y = "vel_x_mu_per_h", x = "condition", order = hue_order, 
                                   #order=order,
                                   box_pairs=[("siScr", "siCdc42"), ("siScr", "siRac1"), ("siCdc42", "siRac1")],
                                   test='t-test_welch', 
                                   #text_format='full',
                                   text_format='star',
                                   loc='inside', verbose=2)

ax.set_ylabel("velocity against flow in microns/h")
ax.set_title("velocity against flow after 2h")
#ax.set_ylim(-6.5,0)

plt.savefig("velocity_against_flow_box_plot.png")
plt.savefig("velocity_against_flow_box_plot.pdf")

In [ ]:
fig, ax = plt.subplots(1, figsize=(10,10))

single_time_point = plot_migration_speeds[plot_migration_speeds["FRAME"]==23]

plot_data = single_time_point.groupby(["condition", "filename"])["vel_mu_per_h"].mean().reset_index()

sns.boxplot(data = plot_data, x = "condition", order = hue_order, y = "vel_mu_per_h" )
sns.swarmplot(data = plot_data, x = "condition", order = hue_order, y = "vel_mu_per_h" , color="black", size =10)
test_results = add_stat_annotation(ax, data=plot_data, y = "vel_mu_per_h", x = "condition", order = hue_order, 
                                   #order=order,
                                   box_pairs=[("siScr", "siCdc42"), ("siScr", "siRac1"), ("siCdc42", "siRac1")],
                                   test='t-test_welch', 
                                   #text_format='full',
                                   #text_format='star',
                                   text_format='simple',
                                   loc='inside', verbose=2)


ax.set_ylabel("velocity in microns/h")
ax.set_title("velocity after 2h")
ax.set_ylim(0,14.5)
plt.savefig("velocity_box_plot.png")
plt.savefig("velocity_box_plot.pdf")

In [ ]:
plot_data = single_time_point.groupby(["condition", "filename"])["vel_mu_per_h"].mean().reset_index()
fig, ax = plt.subplots(1, figsize=(10,10))
sns.boxplot(data = plot_data, x = "condition", y = "vel_mu_per_h" )
sns.swarmplot(data = plot_data, x = "condition", y = "vel_mu_per_h" , color="black", size =10)